In [119]:
import pandas as pd
import json
import nltk
import gensim
from nltk.stem.snowball import SnowballStemmer
import re
import string
from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary

In [120]:
data = pd.read_json("covidtrack_50K.json", lines=True)
data

,_id,text
0,{'$oid': '5e8e2e85e2367bea919ecb12'},DAY-7\n\n#lockdown in #Lagos #Abuja #Nigeria a...
1,{'$oid': '5e8e2e85e2367bea919ecb13'},RT @Astartiel: Now that we know for a fact sad...
2,{'$oid': '5e8e2e85e2367bea919ecb14'},RT @2013Boodicca: The first UK conviction for ...
3,{'$oid': '5e8e2e85e2367bea919ecb15'},RT @PDChina: Made it! A 104-yr-old World War I...
4,{'$oid': '5e8e2e85e2367bea919ecb16'},RT @DarrenPlymouth: UK #coronavirus statistics...
...,...,...
49995,{'$oid': '5e8e2ea5e2367bea919f8e5d'},"RT @arabnews: #PHOTOS: #Wuhan, the #Chinese ci..."
49996,{'$oid': '5e8e2ea5e2367bea919f8e5e'},RT @febottini: 👏👏 @U2 contributes €10m for PPE...
49997,{'$oid': '5e8e2ea5e2367bea919f8e5f'},#HighImpactVlogs: “WTF?! Something VERY Strang...
49998,{'$oid': '5e8e2ea5e2367bea919f8e60'},RT @TeamPelosi: The #coronavirus crisis contin...


In [121]:
text = data['text']
text

0        DAY-7\n\n#lockdown in #Lagos #Abuja #Nigeria a...
1        RT @Astartiel: Now that we know for a fact sad...
2        RT @2013Boodicca: The first UK conviction for ...
3        RT @PDChina: Made it! A 104-yr-old World War I...
4        RT @DarrenPlymouth: UK #coronavirus statistics...
                               ...                        
49995    RT @arabnews: #PHOTOS: #Wuhan, the #Chinese ci...
49996    RT @febottini: 👏👏 @U2 contributes €10m for PPE...
49997    #HighImpactVlogs: “WTF?! Something VERY Strang...
49998    RT @TeamPelosi: The #coronavirus crisis contin...
49999    RT @RealCandaceO: These have been my exact fee...
Name: text, Length: 50000, dtype: object

In [122]:
stopWords = nltk.corpus.stopwords.words('english')
#using snowballstemmer for lemmatization
stemmer = SnowballStemmer("english")

docCount = {}
wordCount = {}

def tokenize(text, i):
    
    # remove all non alphabet  wasn't sure from the assignment if you wanted just special characters 
    #removed or all non alphabet (i.e. numbers), so here i removed all non-alphabet characters
    words = re.sub("[^a-zA-Z]+", ' ', text)
    ## remove extra spaces and tokenize
    words = re.sub('\s+', ' ', text).split(" ")

    
    ## remove stopwords
    temp = [] 
    for r in words: 
        r = r.lower()
        r = r.translate(str.maketrans('', '', string.punctuation))
        r = r.translate(str.maketrans('', '', string.digits))
        #remove emojis/ non ascii chars
        r = r.encode('ascii', 'ignore').decode('ascii')
        if len(r) > 3:
            if not r in stopWords: 
                temp.append(r) 
                
                if r not in wordCount:
                    wordCount[r] = 1
                else:
                    wordCount[r] += 1
                try:
                    if str(i) not in docCount[r]:
                        tempS = docCount[r]
                        docCount[r] = tempS + ', ' + str(i)
                except:
                    docCount[r] = str(i)
    words = temp
    
    ## stem words
    singles = [stemmer.stem(w) for w in words]

    return singles

def tokenSet(text):
    rem = string.punctuation
    pattern = r"[{}]".format(rem)

    chars = ['\n', '\xa0', '\t']
    text = text.str.replace(pattern, '')
    for i in range(len(chars)):
        text = text.replace({chars[i]:''}, regex=True)
    
    tokens = [''] * len(text)
    for i in range(len(text)):
        if (i%5000==0):
            print(str(i) +'/' + str(len(text)))
        tokens[i] = tokenize(str(text[i]), i)
    return tokens

In [123]:
tokens = tokenSet(text)
tokens

0/50000
5000/50000
10000/50000
15000/50000
20000/50000
25000/50000
30000/50000
35000/50000
40000/50000
45000/50000


[['daylockdown',
  'lago',
  'abuja',
  'nigeria',
  'prevent',
  'measur',
  'combat',
  'covid',
  'coronavirusour',
  'team',
  'deci',
  'httpstcoifpawlz'],
 ['astartiel',
  'know',
  'fact',
  'sad',
  'crazi',
  'coronavirus',
  'transfer',
  'even',
  'asymptomat',
  'pet',
  'wildlif'],
 ['boodicca',
  'first',
  'convict',
  'breach',
  'coronavirus',
  'lockdown',
  'restrict',
  'maria',
  'dinou',
  'black',
  'woman',
  'york',
  'fine'],
 ['pdchina',
  'made',
  'yrold',
  'world',
  'veteran',
  'state',
  'oregon',
  'becom',
  'oldest',
  'known',
  'survivor',
  'coronavirus'],
 ['darrenplymouth',
  'coronavirus',
  'statist',
  'case',
  'total',
  'death',
  'total',
  'deadstay',
  'safe',
  'httpstcow'],
 ['fazelehrassouli',
  'regim',
  'alloc',
  'fund',
  'irgc',
  'instead',
  'address',
  'coronavirus',
  'iran'],
 ['ochasouthsudan',
  'togeth',
  'togeth',
  'coronavirus',
  'covid',
  'ssot',
  'southsudan',
  'httpstcofxmx'],
 ['bucksexton',
  'joebiden',


In [124]:
docCount

{'daylockdown': '0, 28062, 28733, 40054',
 'lagos': '0, 14276, 31033, 31368, 31525, 32941, 33275, 33372, 33598, 35724, 35802, 42579, 42705, 43610, 44664',
 'abuja': '0, 46154, 47653',
 'nigeria': '0, 645, 684, 884, 7302, 7303, 9348, 10517, 12377, 13219, 14124, 15710, 26664, 27025, 28093, 30530, 31033, 31368, 31525, 32941, 33275, 33372, 35724, 35802, 35884, 36387, 39076, 39909, 39937, 39970, 40038, 40075, 40275, 40505, 41101, 41841, 42579, 42705, 43229, 43610, 43707, 44076, 44664, 45841, 46154, 47856, 48199',
 'preventive': '0, 15344, 25114, 25196, 30487, 30643',
 'measure': '0, 15043, 15054, 19089, 25114, 25196, 38032',
 'combating': '0, 2930, 2999, 3100, 3610, 3777, 4354, 5156, 5395, 6584, 8869, 9840, 10152, 10954, 13831, 14531, 16434, 22201, 25242, 29937, 31136, 43777, 45177, 47070, 47502, 47654',
 'covid': '0, 6, 25, 36, 61, 68, 77, 80, 88, 91, 102, 109, 119, 123, 130, 136, 137, 145, 148, 154, 165, 167, 171, 174, 179, 181, 187, 194, 196, 234, 243, 245, 252, 260, 267, 279, 292, 293, 

In [125]:
len(wordCount)

47560

In [126]:
newStops = []
for word in docCount:
    count = docCount[word].count(',') + 1
    #print(count)
    if count > (50000*.9) or count < 10 :
        newStops.append(word)
print(len(newStops))
print(len(docCount))


42530
47560


In [127]:
tokensC = tokens
for i in range(len(tokensC)):
    for tempW in tokensC[i]:
        if tempW in newStops:
            tokensC[i].remove(tempW)


In [128]:
common_dictionary = Dictionary(tokensC)
common_corpus = [common_dictionary.doc2bow(token) for token in tokensC]

In [129]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=common_corpus,
                                           id2word=common_dictionary,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [130]:
for i in range(20):
    print(lda_model.print_topics()[i])
    print('...')



(0, '0.288*"https" + 0.090*"go" + 0.068*"discuss" + 0.067*"rememb" + 0.049*"address" + 0.044*"tomorrow" + 0.041*"avoid" + 0.035*"tonight" + 0.020*"strategi" + 0.011*"assist"')
...
(1, '0.633*"covid" + 0.105*"lockdown" + 0.066*"record" + 0.038*"make" + 0.032*"mask" + 0.020*"face" + 0.009*"site" + 0.008*"full" + 0.008*"experi" + 0.004*"requir"')
...
(2, '0.192*"watch" + 0.146*"fight" + 0.090*"piersmorgan" + 0.042*"right" + 0.042*"talk" + 0.035*"learn" + 0.034*"among" + 0.034*"johnson" + 0.033*"condit" + 0.032*"bori"')
...
(3, '0.251*"report" + 0.204*"today" + 0.086*"iran" + 0.045*"bbckatyaadl" + 0.042*"regim" + 0.039*"fear" + 0.038*"centr" + 0.029*"hand" + 0.028*"realli" + 0.027*"fund"')
...
(4, '0.120*"china" + 0.099*"warn" + 0.096*"home" + 0.091*"live" + 0.067*"wuhan" + 0.067*"citi" + 0.048*"stay" + 0.044*"well" + 0.040*"lift" + 0.039*"everyon"')
...
(5, '0.548*"peopl" + 0.077*"staff" + 0.059*"tackl" + 0.047*"ndtv" + 0.030*"ajenglish" + 0.025*"measur" + 0.017*"suggest" + 0.015*"combat"